In [1]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
# Metrics
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
# Text box
import ipywidgets as widgets
from IPython.display import display

# Loading Data

In [2]:
movie_lines_path = 'movie_lines.txt'
movie_conversations_path = 'movie_conversations.txt'

In [25]:
with open(movie_lines_path, encoding='iso-8859-1', errors='ignore') as my_file:
    all_lines = {}
    for line in my_file:
        split = line.split(' +++$+++ ')
        linemp = {}
        fields = ["lineID", "characterID", "movieID", "character", "text"]
        count = 0
        for field in (fields):
                linemp[field] = split[count]
                count +=1
        all_lines[linemp['lineID']] = linemp        
        


In [73]:
with open(movie_conversations_path, encoding='iso-8859-1', errors='ignore') as my_file:
    conv = []
    for line in my_file:
        split = line.split(' +++$+++ ')
        obj = {}
        fields = ["character1ID", "character2ID", "movieID", "utteranceIDs"]
        count = 0 
        for field in fields:
            obj[field] = split[count]
            count +=1
        ID = re.compile('L[0-9]+').findall(obj['utteranceIDs'])
        lines = []
        
        for id_ in ID:
            lines.append(all_lines[id_])
        obj['line'] = lines
        conv.append(obj)

In [74]:
all_lines["L985"]

{'lineID': 'L985',
 'characterID': 'u0',
 'movieID': 'm0',
 'character': 'BIANCA',
 'text': 'I hope so.\n'}

In [75]:
conv[10]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L367', 'L368']\n",
 'line': [{'lineID': 'L367',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': 'How do you get your hair to look like that?\n'},
  {'lineID': 'L368',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': "Eber's Deep Conditioner every two days. And I never, ever use a blowdryer without the diffuser attachment.\n"}]}

# Matching Data

In [91]:
pairs = []
for convrtsation in conv:
        for i in range(len(convrtsation['line'])):
            try:
                question = convrtsation['line'][i]['text'].strip()
                answer = convrtsation['line'][i+1]['text'].strip()
            except:
                pass
            if(question and answer):
                pair = []
                pair.append([question, answer])
                pairs.append(pair)

In [100]:
len(pairs)

304309

In [101]:
for i in range (10):
    print(pairs[i])

[['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.', "Well, I thought we'd start with pronunciation, if that's okay with you."]]
[["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.']]
[['Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]]
[["Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]]
[["You're asking me out.  That's so cute. What's your name again?", 'Forget it.']]
[['Forget it.', 'Forget it.']]
[["No, no, it's my fault -- we didn't have a proper introduction ---", 'Cameron.']]
[['Cameron.', "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."]]
[["The thi

In [ ]:
def Vocab:
    ### This will be the class that handles the bag of words.
    